<a href="https://colab.research.google.com/github/gregnero/byebyespotify/blob/main/byebyespotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bye Bye Spotify ! <3
You've worked so hard on your playlists. Carefully choosing cover pictures, writing angsty and deep descriptions with the hope that ur crush will see them and fall in love with you and run away with you into the Italian countryside ... no? Just me? Awkward...

Whatever the case, the playlist in its entirety is an art form. The cover image and the description are critically important to the integrity and personality of the playlist. Here is a way that you can export all of these things.

This lets you archive and export all of your playlists with a single python script. This way, you can ditch Spotify and rest easy knowing that all of your hard work making playlists will be around in your personal files long after you're gone.

I've tried to make this process as simple as possible but I'm not an app developer so there's lots of room for improvement but this should work fine for now. It would be cool to have a website that does this or a nice app, but a single executable python script seems like a managable place to start. I also tried getting this to run in Google Collab but had some issues... Anyways, let's get started.

# Step 1: Create a Spotify App

The purpose of this step is to create a pipeline by which our code can talk to your Spotify library. This isn't as tricky as it sounds. Go here: https://developer.spotify.com/dashboard and then hit "Create app" . Fill in the following fields:
* App name: this can be whatever. I'd suggest "byespotify"
* App description: this can also be whatever.
* Redirect URI: put in https://example.com/callback
* Make sure you hit "Add"
* Check "understand and agree" box.
* Hit "Save"

Congrats, you've made the app!

On the app page, there will be three crucial pieces of information you need, which you can copy and paste:

* The Client ID
* The Client secret
* The Redirect URI

Hold on to these for later.

# Step 2: Run the Code Below

In [ ]:
# -*- coding: utf-8 -*-

#!/usr/bin/env python3
"""
Spotify Playlist Exporter (Colab-Compatible)
- Manual authentication: copy URL -> authorize -> paste redirected URL
- Exports playlists into directories with tracks.csv, cover.jpg, description.txt
- Separates into "My Playlists" and "Other Playlists"
- Folder names start with date (first track added_at)
- HTML-decoded descriptions
- Per-playlist stats
- Global progress bar
- Zips all folders at the end
- IMPORTANT: Use a registered public Redirect URI, e.g. "https://example.com/callback"
"""

import os
import sys
import time
import zipfile
import shutil
import subprocess
import importlib.util
import re
import html
from datetime import datetime

# ------------------------------
# AUTO-INSTALL DEPENDENCIES
# ------------------------------
REQUIRED_PACKAGES = ["spotipy", "requests", "pandas"]

def install_package(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

def ensure_dependencies():
    for pkg in REQUIRED_PACKAGES:
        if importlib.util.find_spec(pkg) is None:
            print(f"Installing missing package: {pkg} ...")
            install_package(pkg)

ensure_dependencies()

import spotipy
from spotipy.oauth2 import SpotifyOAuth
import requests
import pandas as pd

# ------------------------------
# PROGRESS BAR
# ------------------------------
def progress_bar(current, total, bar_length=30, prefix=""):
    fraction = current / max(total,1)
    filled = int(bar_length * fraction)
    bar = "#" * filled + "-" * (bar_length - filled)
    print(f"\r{prefix}[{bar}] {int(fraction*100)}%", end="", flush=True)

# ------------------------------
# SAFE FILENAME
# ------------------------------
def safe_filename(name):
    if not name:
        name = "untitled"
    return re.sub(r'[<>:"/\\|?*]', '_', name.strip())

# ------------------------------
# ZIP FUNCTION
# ------------------------------
def zip_directory(folder_path, output_zip):
    with zipfile.ZipFile(output_zip, "w", zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                full_path = os.path.join(root, file)
                arcname = os.path.relpath(full_path, folder_path)
                zipf.write(full_path, arcname)

# ------------------------------
# EXPORT FUNCTION
# ------------------------------
def export_playlists(sp, output_root):
    print("\nRetrieving playlists...")
    playlists = []
    results = sp.current_user_playlists(limit=50)
    while results:
        playlists += results['items']
        if results.get("next"):
            results = sp.next(results)
        else:
            break

    print(f"Found {len(playlists)} playlists.\n")
    start_time = time.time()
    total_playlists = len(playlists)
    me = sp.current_user()['id']

    export_folder = os.path.join(output_root, "spotify_export_temp")
    os.makedirs(export_folder, exist_ok=True)

    my_folder = os.path.join(export_folder, "My Playlists")
    other_folder = os.path.join(export_folder, "Other Playlists")
    os.makedirs(my_folder, exist_ok=True)
    os.makedirs(other_folder, exist_ok=True)

    unnamed_counter = 1

    for idx, p in enumerate(playlists, start=1):
        playlist_start = time.time()
        owner_id = p.get("owner", {}).get("id", "")
        base = my_folder if owner_id == me else other_folder

        # Determine playlist folder date prefix
        date_prefix = datetime.today().strftime("%Y-%m-%d")
        items = sp.playlist_items(p.get("id"), limit=1)
        if items.get("items"):
            first_added = items["items"][0].get("added_at")
            if first_added:
                try:
                    date_prefix = datetime.fromisoformat(first_added.replace("Z", "+00:00")).strftime("%Y-%m-%d")
                except:
                    pass

        # Playlist folder
        name = p.get("name") or f"Unnamed Playlist {unnamed_counter}"
        if not p.get("name"):
            unnamed_counter += 1
        name = safe_filename(name)
        playlist_folder = os.path.join(base, f"{date_prefix} - {name}")
        os.makedirs(playlist_folder, exist_ok=True)

        print(f"\n\nStarting export of playlist: '{name}'")

        # Description
        desc = html.unescape(p.get("description") or "")
        with open(os.path.join(playlist_folder, "description.txt"), "w", encoding="utf-8") as f:
            f.write(desc)

        # Cover image
        images = p.get("images") or []
        if images:
            try:
                img_url = images[0].get("url")
                if img_url:
                    img = requests.get(img_url, timeout=10)
                    with open(os.path.join(playlist_folder, "cover.jpg"), "wb") as f:
                        f.write(img.content)
            except:
                pass

        # Tracks
        tracks = []
        results_tracks = sp.playlist_tracks(p.get("id"), limit=100)
        while True:
            for item in results_tracks.get("items", []):
                t = item.get("track")
                if not t:
                    continue
                artists_list = t.get("artists") or []
                artists = ", ".join([str(a.get("name") or "") for a in artists_list if a])
                tracks.append({
                    "Track Name": t.get("name") or "",
                    "Artists": artists,
                    "Album": t.get("album", {}).get("name") or "",
                    "Release Date": t.get("album", {}).get("release_date") or "",
                    "Duration (ms)": t.get("duration_ms") or 0,
                    "Popularity": t.get("popularity") or 0,
                    "Added At": item.get("added_at") or "",
                    "Track URL": t.get("external_urls", {}).get("spotify") or ""
                })
            if results_tracks.get("next"):
                results_tracks = sp.next(results_tracks)
            else:
                break

        df = pd.DataFrame(tracks)
        df.to_csv(os.path.join(playlist_folder, "tracks.csv"), index=False, encoding="utf-8")

        # Playlist timing & stats
        playlist_elapsed = time.time() - playlist_start
        minutes = int(playlist_elapsed // 60)
        seconds = int(playlist_elapsed % 60)
        num_tracks = len(tracks)
        playlist_date = date_prefix
        print(f"Finished exporting playlist '{name}' | Tracks: {num_tracks} | Playlist Creation Date: {playlist_date} | Export Time: {minutes}m {seconds}s")

        # Global progress bar
        progress_bar(idx, total_playlists, prefix="Overall progress: ")

    # ZIP
    zip_date = time.strftime("%Y-%m-%d")
    zip_name = os.path.join(output_root, f"spotify_export_{zip_date}.zip")
    zip_directory(export_folder, zip_name)

    shutil.rmtree(export_folder)

    elapsed = time.time() - start_time
    minutes = int(elapsed // 60)
    seconds = int(elapsed % 60)

    print("\n\n======================================")
    print("🎉 EXPORT COMPLETE!")
    print("======================================")
    print(f"Total playlists exported: {total_playlists}")
    print(f"Total time elapsed: {minutes}m {seconds}s")
    print(f"ZIP file created: {zip_name}")
    print("======================================\n")

# ------------------------------
# ENTRY POINT
# ------------------------------
if __name__ == "__main__":
    print("\n=== Spotify Playlist Export Tool (Colab-Compatible) ===")
    print("\nIMPORTANT: Your Redirect URI must be public and registered in Spotify.")
    print("Example: https://example.com/callback\n")
    client_id = input("Enter your Spotify Client ID: ").strip()
    client_secret = input("Enter your Spotify Client Secret: ").strip()
    redirect_uri = input("Enter your Redirect URI (exactly as registered, e.g., https://example.com/callback): ").strip()

    auth_manager = SpotifyOAuth(
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri=redirect_uri,
        scope="playlist-read-private playlist-read-collaborative",
        open_browser=False
    )

    auth_url = auth_manager.get_authorize_url()
    print("\nOpen this URL in your browser and authorize access:\n")
    print(auth_url)
    redirected_url = input("\nAfter login, paste the full redirected URL here: ").strip()

    code = auth_manager.parse_response_code(redirected_url)
    # Use as_dict=False to get token string directly (DeprecationWarning fixed)
    token_str = auth_manager.get_access_token(code, as_dict=False)
    sp = spotipy.Spotify(auth=token_str)

    output_root = os.getcwd()
    print(f"\nExporting playlists to current directory: {output_root}\n")
    export_playlists(sp, output_root)

# Step 3: Download Your Archived Playlists!

In the bar on the left you'll see a little folder icon and should see your finished .zip file which contains all of your organized playlists, including their cover picture and descriptions. Hit the meatball menu option on that file and download it. That's all ! : )

# Step 4: Find New Ways to Listen to Music

I've been exploring music in the digital world on:
* Bandcamp
* NTS Radio

And other places like:
* The radio
* Live shows
* Physical media (vinyls, casettes)

Listen to music your pals make ! There's so much good music in your community : )

I encourage you to check out any platforms that aren't actively supporting the fascist regime and platforms that compensate their musicians fairly. <3